In [0]:
from pyspark.sql.types import StructType, StructField, StringType, DoubleType, DateType, LongType 
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
import requests
from datetime import datetime
from pyspark.sql.functions import when, col, round, max

In [0]:
def get_cdi_today():
    url = "https://brasilapi.com.br/api/taxas/v1/cdi"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()
        cdi = 1 + (data['valor'] / 100)
        return cdi
    except requests.exceptions.RequestException as e:
        print(f"Erro ao conectar à API da Brasil API: {e}")
        return None
    except (ValueError, IndexError) as e:
        print(f"Erro ao processar os dados da API: {e}")
        return None

cdi_data = get_cdi_today()

In [0]:
def carregando_ultima_partition(nome_tabela, coluna_particao):

    schema = StructType([
        StructField("codigo", StringType(), True),
        StructField("origem", StringType(), True),
        StructField("extract", StringType(), True),
        StructField("desconto", StringType(), True),
        StructField("link", StringType(), True),
        StructField("nome", StringType(), True),
        StructField("parcelamento", StringType(), True),
        StructField("preco", StringType(), True),
        StructField("data_ref", StringType(), True)
    ])

    print(f"Tentando ler arquivo Delta de: {nome_tabela}")
    
    spark = SparkSession.builder.getOrCreate()
    dbutils = DBUtils(spark) 

    # 1- Listando arquivos Delta no Volume Bronze
    print(f"\n--- Verificação do caminho Volume Bronze ---")
    try:
        dbutils.fs.ls(nome_tabela)
        print(f"STATUS OK - Caminho '{nome_tabela}' existe. Tentando carregar como Delta.")
    except Exception as e:
        print(f"STATUS ALERTA - O caminho '{nome_tabela}' não existe ou não é acessível. Detalhe: {e}. Gerando um DataFrame vazio.")
        print("\nRetornando um dataframe vazio!")
        # Em caso de erro, retorne um dataframe vazio com o schema definido anteriormente
        return spark.createDataFrame([], schema=schema)

    # 2- Tentando ler o arquivo Delta do Volume Bronze
    print(f"\n--- Verificação da leitura do arquivo Delta ---")
    try:
        df = spark.read.format("delta").load(nome_tabela)
        print(f"STATUS OK - Tabela Delta '{nome_tabela}' carregada com sucesso.")
    except Exception as e:
        # Em caso de erro, retorne um dataframe vazio com o schema definido anteriormente
        print(f"STATUS ALERTA - Erro ao carregar a tabela Delta '{nome_tabela}'. Provavelmente não é uma tabela Delta válida ou não contém dados. Detalhe do erro: {e}")
        print("\nRetornando um dataframe vazio!")
        return spark.createDataFrame([], schema=schema)
    
    # 3- Verificando se o dataframe está vazio
    # Em caso positivo, retorne um dataframe vazio com o schema definido anteriormente
    print(f"\n--- Verificação se arquivo Delta está vazio ---")
    if df.rdd.isEmpty():
        print(f"STATUS ALERTA - Tabela '{nome_tabela}' foi carregada como Delta VÁLIDA, mas está completamente vazia. Retornando DataFrame vazio com o schema definido.")
        print("\nRetornando um dataframe vazio!")
        return spark.createDataFrame([], schema=schema)
    else:
        print(f"STATUS OK - Tabela Delta '{nome_tabela}' não está vazio.")

    # 4- Buscando a ultima partição do dataframe
    print(f"\n--- Verificando ultima partição ---")
    ultima_particao_df = df.select(max(coluna_particao).alias("ultima_particao"))
    ultima_particao = None
    if ultima_particao_df.first():
        print(f"STATUS OK - Tabela Delta '{nome_tabela}' contém partição: {ultima_particao_df}.")
        ultima_particao = ultima_particao_df.first()["ultima_particao"]

    print(f"\n--- Filtrando Tabela pela ultima partição ---")
    # 5- Iniciando filtro da ultima partição da tabeça
    if ultima_particao is not None:
        filtro = f"{coluna_particao} = '{ultima_particao}'"
        df_ultima_particao = df.where(filtro)
        print(f"STATUS OK - Tabela '{nome_tabela}' filtrada pela última partição: {ultima_particao}")

        # Realizando contagem de linhas do dataframe filtrado
        qtd = df_ultima_particao.count()

        # Se o dataframe estiver vazio, ou seja, menor que 1 linha, retorne um erro pois o dataframe não deve estar vazio
        assert qtd > 0, f"A última partição '{ultima_particao}' da tabela '{nome_tabela}' está vazia."
        
        print(f"Leitura da tabela '{nome_tabela}' carregada com sucesso. Número de linhas: {qtd}")

        return df_ultima_particao
    # Em caso de nenhuma partição encontrada no dataframe, será retornado um dataframe vazio com o schema definido anteriormente
    else:
        print(f"STATUS ALERTA - Não foram encontradas partições na tabela '{nome_tabela}'.")
        return spark.createDataFrame([], schema=df.schema)

# Caminho para a external location do diretório silver
silver_path = f"/Volumes/nintendodatabrickspkjgt7_workspace/nintendo/silver"

# Lendo arquivo Delta do diretório bronze pela ultima partição
df = carregando_ultima_partition(silver_path, "data_ref")

In [0]:
def recomendation_price(df, cdi_data):

    df_price_desconto = df.withColumn("preco_desconto", round(col("preco") - (col("preco") * col("desconto")), 2))

    df_price_parcelado = df_price_desconto.withColumn("preco_parcelado", round((col("valor_prestacao") * col("numero_parcelas")), 2))

    df_save_desconto = df_price_parcelado.withColumn("save_desconto", round((col("preco") - col("preco_desconto")), 2))

    df_save_parcelado = df_save_desconto.withColumn(
        "save_parcelado",
        when((col("preco") - col("preco_parcelado")) * cdi_data <= 0, 0)
        .otherwise(round(((col("preco") - col("preco_parcelado")) * cdi_data), 2))
    )

    df_pct_parcelado = df_save_parcelado.withColumn(
        "pct_preco_parcelado",
        round(100 -((col("preco_parcelado") / col("preco")) * 100), 2)
    )

    df_recomendacao = df_pct_parcelado.withColumn(
        "recomendacao",
        when(col("desconto") < col("pct_preco_parcelado"), "Comprar Parcelado")
        .when(col("desconto") > col("pct_preco_parcelado"), "Comprar a Vista Pix")
        .otherwise("Sem Vantagem Identificada")
    ).drop("preco_desconto", "preco_parcelado", "save_desconto", "save_parcelado", "pct_preco_parcelado")

    return df_recomendacao

df_recomendacao = recomendation_price(df, cdi_data)

In [0]:
def carregando_tabela_gold(df,delta_table_path):

    print(f"Iniciando o salvamento do DataFrame no formato Delta em: {delta_table_path}")

    try:
        # 1- Obter a contagem de linhas ANTES de salvar ---
        num_rows_to_save = df.count()
        print(f"Número de linhas no DataFrame a ser salvo: {num_rows_to_save}")

        # 2- Salvar o DataFrame no formato Delta ---
        df.write \
                        .format("delta") \
                        .mode("overwrite") \
                        .partitionBy("data_ref") \
                        .save(delta_table_path)

        print(f"DataFrame salvo com sucesso como tabela Delta particionada por 'extract' em: {delta_table_path}")

        # Início das Verificações de Qualidade Pós-Gravação 

        # 3- Garantir que os dados foram salvos no caminho
        print(f"\n--- Verificação: Leitura da Tabela Delta Salva ---")
        df_delta_read = spark.read.format("delta").load(delta_table_path)
        print("Esquema da tabela Delta lida:")
        df_delta_read.printSchema()
        num_rows_saved = df_delta_read.count()

        if df_delta_read.isEmpty():
            print(f"STATUS ALERTA - A tabela Delta salva em '{delta_table_path}' está vazia ou não pôde ser lida.")
        else:
            print(f"STATUS OK - A tabela Delta foi lida com sucesso de '{delta_table_path}' com {num_rows_saved} linhas recarregadas.")


    except Exception as e:
        print(f"Ocorreu um erro geral ao salvar ou verificar a tabela Delta: {e}")


# Caminho para a external location do diretório gold
gold_path = f"/Volumes/nintendodatabrickspkjgt7_workspace/nintendo/gold"

# Sobreescrevendo dados particionados no diretório gold
carregando_tabela_gold(df_recomendacao, gold_path)